# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import re
from sqlalchemy import create_engine
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Disasters', con=engine)
X = df['message']
y = df.iloc[:, 4:39]
y['related'].replace(2, 1, inplace=True)
df.head(10)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [3]:
#import libraries
import nltk
import re
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to C:\Users\Ahmed Sayed
[nltk_data]     Mansour\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ahmed Sayed
[nltk_data]     Mansour\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#functio def
def tokenize(data):
    data = re.sub(r'[^\w\s]','',data)
    tokenization = word_tokenize(data)
    lemmatizers = WordNetLemmatizer()

    cleaned = []
    for tok in tokenization:
        clean_tok = lemmatizers.lemmatize(tok).lower().strip()
        cleaned.append(clean_tok)

    return cleaned

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
#imports
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F2D1DC89D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

In [10]:
import numpy as np
print(pipeline.predict( np.array([X_test.iloc[0]]) ))

[[1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
print(classification_report(y_test.iloc[:,0], y_pred[:,0]))

              precision    recall  f1-score   support

           0       0.73      0.28      0.41      1479
           1       0.82      0.97      0.89      5075

    accuracy                           0.81      6554
   macro avg       0.78      0.63      0.65      6554
weighted avg       0.80      0.81      0.78      6554



In [15]:
evaluation = {}
for column in y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(recall_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(f1_score(y_test[column], y_pred_df[column]))

pd.DataFrame(evaluation)

C:\Users\Ahmed Sayed Mansour\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ahmed Sayed Mansour\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ahmed Sayed Mansour\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,0.822635,0.878049,0.0,0.786285,0.693878,0.689655,0.545455,0.0,0.812500,0.0,...,0.0,0.0,0.0,0.85495,0.896907,0.766272,0.0,0.877159,1.000000,0.736842
1,0.969655,0.422764,0.0,0.618397,0.063551,0.060423,0.033520,0.0,0.058036,0.0,...,0.0,0.0,0.0,0.62479,0.332061,0.422512,0.0,0.764214,0.074074,0.041791
2,0.890115,0.570732,0.0,0.692308,0.116438,0.111111,0.063158,0.0,0.108333,0.0,...,0.0,0.0,0.0,0.72197,0.484680,0.544690,0.0,0.816801,0.137931,0.079096


In [17]:
for i in range(35):
    print("Precision, Recall, F1 Score for {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Precision, Recall, F1 Score for related
              precision    recall  f1-score   support

           0       0.73      0.28      0.41      1479
           1       0.82      0.97      0.89      5075

    accuracy                           0.81      6554
   macro avg       0.78      0.63      0.65      6554
weighted avg       0.80      0.81      0.78      6554

Precision, Recall, F1 Score for request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5447
           1       0.88      0.42      0.57      1107

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.75      6554
weighted avg       0.89      0.89      0.88      6554

Precision, Recall, F1 Score for offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6554
   macro a

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
from sklearn.model_selection import GridSearchCV

parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False)
    }
cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print("Accuracy")
print((y_pred == y_test).mean())
for i in range(35):
    print("Precision, Recall, F1 Score for {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
from sklearn.ensemble import AdaBoostClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


In [22]:
from sklearn.metrics import classification_report
for i in range(35):
    print("Precision, Recall, F1 Score for {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Precision, Recall, F1 Score for related
              precision    recall  f1-score   support

           0       0.62      0.33      0.43      1479
           1       0.83      0.94      0.88      5075

    accuracy                           0.80      6554
   macro avg       0.73      0.64      0.66      6554
weighted avg       0.78      0.80      0.78      6554

Precision, Recall, F1 Score for request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5447
           1       0.79      0.53      0.63      1107

    accuracy                           0.90      6554
   macro avg       0.85      0.75      0.79      6554
weighted avg       0.89      0.90      0.89      6554

Precision, Recall, F1 Score for offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6554
   macro a

### 9. Export your model as a pickle file

In [25]:
import pickle
filename = 'models/model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.